
# 🎓 초급 딥러닝 미션:
# 딥러닝 학교 – 마법 신경망으로 세계를 구하라!

---

## 🏰 배경 스토리

당신은 **AI 마법학교의 입학생**입니다.  
이 학교에서는 사람의 감정, 날씨, 질병을 예측하는 **마법 신경망**을 수련합니다.  
당신의 첫 수업은 **딥러닝 기초 수련**입니다.  
당신은 NumPy만으로 **마법 신경망**을 직접 구현하고, 다양한 마법서(Optimizer)를 실험하며,  
마지막엔 **PyTorch 마법 시스템**에 입문하게 됩니다.

---

## 🎯 미션 학습 목표

- 딥러닝 구조 및 원리 이해 (forward, loss, backward)
- 학습률 개념 실험
- 옵티마이저 종류 비교 (SGD, Momentum, RMSProp, Adam)
- 파이토치 기초 문법 학습 및 비교

---

## 🧩 미션 구성



### ✅ Part 1. 딥러닝 개요 퀴즈

1. 딥러닝에서 '신경망'이란 무엇을 모방한 것인가요?  
   a. 컴퓨터 회로  
   b. 인간 뇌의 뉴런 구조  
   c. 수학 공식  
   d. 로봇 제어기

2. Backpropagation은 무엇을 계산하는 과정인가요?  
   a. 데이터 전처리  
   b. 예측 결과 시각화  
   c. 손실 함수의 그래디언트를 역방향으로 계산  
   d. 예측값을 정규화하는 함수


b,c


### ✅ Part 2. NumPy 기반 신경망 실습

#### 🎯 문제: XOR 문제를 NumPy만으로 푸는 2층 신경망을 직접 구현해보세요.

- 입력: 2차원 벡터 (ex: [0,1], [1,1])
- 출력: 0 또는 1
- 구조: 입력(2) → 은닉층(4, ReLU) → 출력층(1, Sigmoid)
- 손실 함수: Binary Cross Entropy

#### ✏️ 구현 항목
1. forward() – 순전파 계산
2. compute_loss() – 손실 계산
3. backward() – 역전파 계산 (Chain Rule 기반)
4. update_weights() – SGD 기반 파라미터 갱신 ==> 중요함!!
5. 학습률 변화 실험 (0.001 / 0.01 / 0.1)


In [ ]:
import numpy as np

class XORNetwork:
    def __init__(self, input_size=2, hidden_size=4):
        # 가중치 초기화
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1)
        self.b2 = np.zeros((1, 1))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(0, x)

    def forward(self, X):
        # 순전파
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def compute_loss(self, Y, Y_pred):
        # Binary Cross Entropy 손실
        return -np.mean(Y * np.log(Y_pred) + (1 - Y) * np.log(1 - Y_pred))

    def backward(self, X, Y):
        # 역전파 구현
        m = X.shape[0]

        # 출력층 오차
        dZ2 = self.a2 - Y
        dW2 = (1/m) * np.dot(self.a1.T, dZ2)
        db2 = (1/m) * np.sum(dZ2, axis=0)

        # 은닉층 오차
        dZ1 = np.dot(dZ2, self.W2.T) * (self.z1 > 0)
        dW1 = (1/m) * np.dot(X.T, dZ1)
        db1 = (1/m) * np.sum(dZ1, axis=0)

        return dW1, db1, dW2, db2

    def update_weights(self, dW1, db1, dW2, db2, learning_rate=0.01):
        # SGD 가중치 업데이트
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

# 학습
X = np.array([[0,0], [0,1], [1,0], [1,1]])
Y = np.array([[0], [1], [1], [0]])

model = XORNetwork()
epochs = 10000

for epoch in range(epochs):
    # 순전파
    Y_pred = model.forward(X)

    # 손실 계산
    loss = model.compute_loss(Y, Y_pred)

    # 역전파
    dW1, db1, dW2, db2 = model.backward(X, Y)

    # 가중치 업데이트
    model.update_weights(dW1, db1, dW2, db2)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss}")

Epoch 0, Loss: 1.093064599598056
Epoch 1000, Loss: 0.6952855339915367
Epoch 2000, Loss: 0.6936971588677859
Epoch 3000, Loss: 0.6932910022123189
Epoch 4000, Loss: 0.6931849512003239
Epoch 5000, Loss: 0.6931571110298702
Epoch 6000, Loss: 0.6931497921969233
Epoch 7000, Loss: 0.6931478674533718
Epoch 8000, Loss: 0.6931473612252397
Epoch 9000, Loss: 0.6931472280784128



### ✅ Part 3. 다양한 옵티마이저 실험 (NumPy 구현 or 의사코드)

| Optimizer | 설명 |
|-----------|------|
| SGD | 가장 기본적인 경사하강법 |
| Momentum | 이전 기울기를 누적하여 반동 효과 부여 |
| RMSProp | 각 파라미터별 적응적 학습률 적용 |
| Adam | Momentum + RMSProp을 결합한 대표적 옵티마이저 |


아래 코드는 어떤 방식의 옵티마이저의 구현인지 답하시오.


In [ ]:
# 💡 간단한 옵티마이저 의사코드 예시임.
v = 0
v = beta * v + (1 - beta) * grad
w = w - learning_rate * v


모멘텀 옵티마이저


### ✅ Part 4. PyTorch 입문 과제

이제부터는 AI 마법학교의 공식 프레임워크인 **PyTorch**를 배웁니다!

#### 🎯 미션: 위 XOR 문제를 PyTorch로 다시 풀어보세요.
- PyTorch로 간단한 학습 루프 예제 코드를 검색해서 찾아서 직접 작성해보세요.
- torch.nn.Linear, torch.ReLU, torch.Sigmoid, torch.BCELoss 등 사용
- torch.optim.SGD, Adam, RMSprop 등 Optimizer로 비교


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 데이터 준비
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# 모델 정의
class XORNetwork(nn.Module):
    def __init__(self):
        super(XORNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 4),  # 입력층 2개 → 은닉층 4개
            nn.ReLU(),         # ReLU 활성화 함수
            nn.Linear(4, 1),   # 은닉층 4개 → 출력층 1개
            nn.Sigmoid()       # 이진 분류를 위한 시그모이드
        )

    def forward(self, x):
        return self.model(x)

# 모델, 손실함수, 옵티마이저 설정
model = XORNetwork()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 루프
epochs = 5000
for epoch in range(epochs):
    # 순전파
    outputs = model(X)

    # 손실 계산
    loss = criterion(outputs, Y)

    # 역전파 및 가중치 갱신
    optimizer.zero_grad()  # 기울기 초기화
    loss.backward()        # 역전파
    optimizer.step()       # 가중치 업데이트

    # 일정 주기로 손실 출력
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 최종 예측 및 정확도 평가
with torch.no_grad():
    predictions = (model(X) > 0.5).float()
    accuracy = (predictions == Y).float().mean()
    print(f'\n최종 정확도: {accuracy.item() * 100:.2f}%')

    # 결과 출력
    print("\n예측 결과:")
    for i in range(len(X)):
        print(f"입력: {X[i].numpy()}, 실제값: {Y[i].numpy()}, 예측값: {predictions[i].numpy()}")

Epoch [500/5000], Loss: 0.0221
Epoch [1000/5000], Loss: 0.0061
Epoch [1500/5000], Loss: 0.0028
Epoch [2000/5000], Loss: 0.0016
Epoch [2500/5000], Loss: 0.0010
Epoch [3000/5000], Loss: 0.0007
Epoch [3500/5000], Loss: 0.0005
Epoch [4000/5000], Loss: 0.0003
Epoch [4500/5000], Loss: 0.0002
Epoch [5000/5000], Loss: 0.0002

최종 정확도: 100.00%

예측 결과:
입력: [0. 0.], 실제값: [0.], 예측값: [0.]
입력: [0. 1.], 실제값: [1.], 예측값: [1.]
입력: [1. 0.], 실제값: [1.], 예측값: [1.]
입력: [1. 1.], 실제값: [0.], 예측값: [0.]



## 🏁 마무리 퀴즈

1. 학습률이 너무 크면 발생할 수 있는 문제는?
   - a. 과적합  
   - b. 손실 발산  
   - c. 더 정밀한 학습  
   - d. 정확도가 무조건 올라감

2. Adam 옵티마이저는 어떤 두 가지 개념을 결합한 것인가요?
   - a. LSTM + GRU  
   - b. SGD + Dropout  
   - c. Momentum + RMSProp  
   - d. SGD + AdaGrad


b, c


## 🎁 보너스 미션 (선택)

- 직접 XOR이 아닌 make_moons, make_circles 등의 데이터셋으로 실험해보세요.
- 은닉층의 노드 수를 바꿔가며 학습 성능을 비교해보세요.



In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import make_moons, make_circles
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 데이터셋 생성 및 전처리 함수
def prepare_dataset(dataset_type='moons', test_size=0.2):
    # 데이터셋 생성
    if dataset_type == 'moons':
        X, y = make_moons(n_samples=500, noise=0.15, random_state=42)
    else:  # circles
        X, y = make_circles(n_samples=500, noise=0.1, random_state=42)

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )

    # 표준화
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # NumPy → PyTorch 텐서 변환
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)
    y_train = torch.FloatTensor(y_train).unsqueeze(1)
    y_test = torch.FloatTensor(y_test).unsqueeze(1)

    return X_train, X_test, y_train, y_test

# 신경망 모델 클래스
class FlexibleNetwork(nn.Module):
    def __init__(self, input_size=2, hidden_sizes=[4, 4], output_size=1):
        super(FlexibleNetwork, self).__init__()

        # 동적으로 레이어 생성
        layers = []
        prev_size = input_size

        # 은닉층 추가
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size

        # 출력층
        layers.append(nn.Linear(prev_size, output_size))
        layers.append(nn.Sigmoid())

        # 순차 모델 생성
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# 학습 및 평가 함수
def train_and_evaluate(X_train, X_test, y_train, y_test, hidden_sizes=[4, 4], lr=0.01, epochs=1000):
    # 모델, 손실함수, 옵티마이저 설정
    model = FlexibleNetwork(hidden_sizes=hidden_sizes)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # 학습
    for epoch in range(epochs):
        # 순전파
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 평가
    with torch.no_grad():
        test_outputs = model(X_test)
        predictions = (test_outputs > 0.5).float()
        accuracy = (predictions == y_test).float().mean()

    return accuracy.item()

# 데이터셋별, 은닉층 구조별 실험
def experiment():
    # 실험할 데이터셋 유형
    datasets = ['moons', 'circles']

    # 실험할 은닉층 구조들
    hidden_layer_configs = [
        [4],           # 1개 은닉층, 4노드
        [4, 4],        # 2개 은닉층, 각 4노드
        [8],           # 1개 은닉층, 8노드
        [8, 8],        # 2개 은닉층, 각 8노드
        [16],          # 1개 은닉층, 16노드
    ]

    # 결과 저장
    results = {}

    # 각 데이터셋에 대해 실험
    for dataset in datasets:
        print(f"\n{dataset.upper()} 데이터셋 실험:")

        # 데이터 준비
        X_train, X_test, y_train, y_test = prepare_dataset(dataset)

        # 각 은닉층 구조로 실험
        dataset_results = {}
        for config in hidden_layer_configs:
            accuracy = train_and_evaluate(
                X_train, X_test, y_train, y_test,
                hidden_sizes=config
            )
            dataset_results[str(config)] = accuracy
            print(f"은닉층 {config}: 정확도 {accuracy*100:.2f}%")

        results[dataset] = dataset_results

    return results

# 실험 및 시각화
results = experiment()


MOONS 데이터셋 실험:
은닉층 [4]: 정확도 85.00%
은닉층 [4, 4]: 정확도 99.00%
은닉층 [8]: 정확도 100.00%
은닉층 [8, 8]: 정확도 100.00%
은닉층 [16]: 정확도 100.00%

CIRCLES 데이터셋 실험:
은닉층 [4]: 정확도 87.00%
은닉층 [4, 4]: 정확도 83.00%
은닉층 [8]: 정확도 85.00%
은닉층 [8, 8]: 정확도 85.00%
은닉층 [16]: 정확도 85.00%
